# ローテーション

In [15]:
%run 0.2_Function.ipynb

In [16]:
df = result_df("2015", "2024", ["東"])

In [17]:
df = df[
    ["レース日", "馬名", "場所", "確定着順", "単勝オッズ", "距離", "芝・ダ", "騎手名",'通過順4', '父馬名']
]

In [18]:
# 距離ローテ
df = df.sort_values(["馬名", "レース日"])

df["前走距離"] = (
    df
    # ← PARTITION BY 馬名
    .groupby("馬名")["距離"]
    # ← LAG(distance, 1)
    .shift(1)
)
df["距離延長"] = (df["距離"] >= df["前走距離"] + 200).fillna(False)
df["距離短縮"] = (df["距離"] <= df["前走距離"] - 200).fillna(False)

In [19]:
# 芝・ダートのローテ
df["前走芝・ダ"] = (
    df
    # PARTITION BY 馬名
    .groupby("馬名")["芝・ダ"]
    # LAG(芝・ダ, 1)
    .shift(1)
)

df["芝→ダ"] = np.where(
    df["前走芝・ダ"].notna()
    & (df["芝・ダ"] != df["前走芝・ダ"])
    & (df["前走芝・ダ"] == "芝"),
    True,  # 前走と違う ⇒ 変更あり
    False,  # 前走と同じ or 前走なし
)
df["ダ→芝"] = np.where(
    df["前走芝・ダ"].notna()
    & (df["芝・ダ"] != df["前走芝・ダ"])
    & (df["前走芝・ダ"] == "ダ"),
    True,  # 前走と違う ⇒ 変更あり
    False,  # 前走と同じ or 前走なし
)

In [22]:
# 騎手のローテ
df["前走騎手名"] = df.groupby("馬名")["騎手名"].shift(1)

df["乗り替わり"] = np.where(
    df["前走騎手名"].notna() & (df["騎手名"] != df["前走騎手名"]), True, False
)

# 判定したい騎手名をリストで用意
top_jockeys = ["ルメール", "レーン", "モレイラ","Ｃ．デム"]

# True / False のブール Series を作成
is_top_jockey = df["前走騎手名"].isin(top_jockeys)
is_not_top_jockey = ~df["騎手名"].isin(top_jockeys)


df["鞍上弱化"] = np.where(
    df["前走騎手名"].notna() & (df["騎手名"] != df["前走騎手名"]) & is_top_jockey & is_not_top_jockey, True, False
)

In [23]:
# 前走通過順
df["前走通過順4"] = (
    df
    # PARTITION BY 馬名
    .groupby("馬名")["通過順4"]
    # LAG(芝・ダ, 1)
    .shift(1)
)

In [24]:
df.head(20)

,レース日,馬名,場所,確定着順,単勝オッズ,距離,芝・ダ,騎手名,通過順4,父馬名,前走距離,距離延長,距離短縮,前走芝・ダ,芝→ダ,ダ→芝,前走騎手名,乗り替わり,鞍上弱化,前走通過順4
75081,2023-10-22,アアル,東京,14,14.7,1400,ダ,菅原明良,12,Malibu Moon,NaN,False,False,NaN,False,False,NaN,False,False,NaN
6170,2015-06-13,アイアイエンデバー,東京,10,65.7,2100,ダ,西田雄一,8,ゴールドヘイロー,NaN,False,False,NaN,False,False,NaN,False,False,NaN
27589,2015-10-18,アイアイエンデバー,東京,12,76.3,2100,ダ,田中勝春,16,ゴールドヘイロー,2100.0,False,False,ダ,False,False,西田雄一,True,False,8.0
28235,2015-11-01,アイアイエンデバー,東京,15,221.9,1400,ダ,江田照男,15,ゴールドヘイロー,2100.0,False,True,ダ,False,False,田中勝春,True,False,16.0
23524,2016-04-30,アイアイエンデバー,東京,12,474.7,1600,芝,嘉藤貴行,9,ゴールドヘイロー,1400.0,True,False,ダ,False,True,江田照男,True,False,15.0
24791,2016-05-22,アイアイエンデバー,東京,9,261.0,1800,芝,嘉藤貴行,9,ゴールドヘイロー,1600.0,True,False,芝,False,False,嘉藤貴行,False,False,9.0
20325,2016-06-04,アイアイエンデバー,東京,9,222.2,2000,芝,西田雄一,7,ゴールドヘイロー,1800.0,True,False,芝,False,False,嘉藤貴行,True,False,9.0
2562,2016-11-26,アイアイハッピー,東京,7,173.8,1600,ダ,石橋脩,8,ゴールドヘイロー,NaN,False,False,NaN,False,False,NaN,False,False,NaN
55555,2017-01-29,アイアイハッピー,東京,8,144.8,1300,ダ,柴山雄一,12,ゴールドヘイロー,1600.0,False,True,ダ,False,False,石橋脩,True,False,8.0
15225,2017-05-21,アイアイハッピー,東京,11,192.5,1600,ダ,野中悠太,5,ゴールドヘイロー,1300.0,True,False,ダ,False,False,柴山雄一,True,False,12.0


In [33]:
df["前走通過順4"] = pd.to_numeric(df["前走通過順4"], errors="coerce")
df["単勝オッズ"]   = pd.to_numeric(df["単勝オッズ"],   errors="coerce")
df["確定着順"]     = pd.to_numeric(df["確定着順"],     errors="coerce")

# フィルタ条件
cond = (
    #(df["前走通過順4"] >= 9) &     # 前走通過順が 9 以上
    #(df["前走通過順4"] <= 4) &     # 前走通過順が 9 以上
    #(df["距離延長"]) &            # 距離延長 == True
    #(df["距離短縮"]) &            # 距離延長 == True
    (df["単勝オッズ"] < 10)   &      # 単勝オッズ < 40
    (df["鞍上弱化"])
)

sub = df.loc[cond].copy()

# 父馬名ごとに集計
summary = (
    sub.groupby("父馬名")
       .agg(
            sample=("確定着順", "size"),
            勝率=("確定着順", lambda s: (s.eq(1)).mean() * 100),
            複勝率=("確定着順", lambda s: (s.le(3)).mean() * 100),
       )
       .round({"勝率": 2, "複勝率": 2})
       .sort_values("sample", ascending=False) 
)
# min sample
min_samples = 35

summary_filtered = summary[summary["sample"] >= min_samples]

summary_filtered.reset_index().sort_values('複勝率', ascending=False)

,父馬名,sample,勝率,複勝率
3,ロードカナロア,44,22.73,54.55
0,ディープインパクト,123,12.20,51.22
1,ハーツクライ,54,18.52,46.30
2,キングカメハメハ,52,17.31,42.31
